Ссылка на гит репозиторий https://github.com/nbasina/project_git

 
<h3> Описание проекта <a class="tocSkip"> </h3>

Цель: выявить сегмент покупателей, у которых предполагается снижение покупательской активности, чтобы сделать им
    персональные предложения.
    
Цель проекта: Обучить 4 модели. Выбрать лучшую модель с наиболее высокой метрикой качества.
    Провести анализ важности признаков лучшей модели и сформировать сегмент пользователей и предложить способы
    увеличения покупательской активности для этого сегмента. 
    
План проекта: 
1. Изучить 4 датасета
2. Провести предобработку данных 
3. Провести исследовательский анализ 
4. Объединить датасеты
5. Проведите корреляционный анализ 
6. Обучить 4 модели используя pipeline  
7. Провести анализ важности признаков
8. Провести сегментацию покупателей и предложить решение
9. Общие выводы


In [1]:

pip install shap

  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/5f/9e/dce41d5ec9e79add65faf4381d8d4492247b29daaa6cc7d7fd0298abc1e2/shap-0.46.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for slicer==0.0.8 from https://files.pythonhosted.org/packages/63/81/9ef641ff4e12cbcca30e54e72fb0951a2ba195d0cda0ba4100e532d929db/slicer-0.0.8-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 485.9 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install phik

  Obtaining dependency information for phik from https://files.pythonhosted.org/packages/b9/69/a42d9320b68b0d43d78db46787669b2287cba5e8db2e83fe0d63c1fd6945/phik-0.12.4-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.3/657.3 kB 544.0 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install -U scikit-learn  

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/cd/7a/19fe32c810c5ceddafcfda16276d98df299c8649e24e84d4f00df4a91e01/scikit_learn-1.5.2-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/4b/2c/ffbf7a134b9ab11a67b0cf0726453cedd9c5043a4fe7a35d1cefa9a1bcfb/threadpoolctl-3.5.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 2.9 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 2.2.0
    Uninstalling threadpoolctl-2.2.0:
      Successfully uninstalled threadpoolctl-2.2.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0


In [3]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from scipy import stats as st
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split,RandomizedSearchCV 
from sklearn.linear_model import (LinearRegression, LogisticRegression)
from sklearn.preprocessing import (OneHotEncoder,StandardScaler,OrdinalEncoder, MinMaxScaler, RobustScaler)
import phik
from phik import phik_matrix
from phik.report import plot_correlation_matrix

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

import shap
from sklearn.feature_selection import SelectKBest, f_classif
import re
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder


In [4]:
RANDOM_STATE = 42
TEST_SIZE = 0.25

## Загрузка данных

In [5]:
market_file = pd.read_csv('market_file - market_file.csv')
display(market_file.head())
print(market_file.info())


,id,Покупательская активность,Тип сервиса,Разрешить сообщать,Маркет_актив_6_мес,Маркет_актив_тек_мес,Длительность,Акционные_покупки,Популярная_категория,Средний_просмотр_категорий_за_визит,Неоплаченные_продукты_штук_квартал,Ошибка_сервиса,Страниц_за_визит
0,215348,Снизилась,премиум,да,3.4,5,121,0.00,Товары для детей,6,2,1,5
1,215349,Снизилась,премиум,да,4.4,4,819,0.75,Товары для детей,4,4,2,5
2,215350,Снизилась,стандартт,нет,4.9,3,539,0.14,Домашний текстиль,5,2,1,5
3,215351,Снизилась,стандартт,да,3.2,5,896,0.99,Товары для детей,5,0,6,4
4,215352,Снизилась,стандартт,нет,5.1,3,1064,0.94,Товары для детей,3,2,3,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id                                   1300 non-null   int64  
 1   Покупательская активность            1300 non-null   object 
 2   Тип сервиса                          1300 non-null   object 
 3   Разрешить сообщать                   1300 non-null   object 
 4   Маркет_актив_6_мес                   1300 non-null   float64
 5   Маркет_актив_тек_мес                 1300 non-null   int64  
 6   Длительность                         1300 non-null   int64  
 7   Акционные_покупки                    1300 non-null   float64
 8   Популярная_категория                 1300 non-null   object 
 9   Средний_просмотр_категорий_за_визит  1300 non-null   int64  
 10  Неоплаченные_продукты_штук_квартал   1300 non-null   int64  
 11  Ошибка_сервиса                

В таблице 1300 строк, без пропусков

In [6]:
market_money = pd.read_csv('market_money - market_money.csv')
display(market_money.head())
print(market_money.info())

,id,Период,Выручка
0,215348,препредыдущий_месяц,0.0
1,215348,текущий_месяц,3293.1
2,215348,предыдущий_месяц,0.0
3,215349,препредыдущий_месяц,4472.0
4,215349,текущий_месяц,4971.6


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       3900 non-null   int64  
 1   Период   3900 non-null   object 
 2   Выручка  3900 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 91.5+ KB
None


В таблице 3900 строк, без пропусков

In [7]:
market_time = pd.read_csv('market_time - market_time.csv')
display(market_time.head())
print(market_time.info())

,id,Период,минут
0,215348,текущий_месяц,14
1,215348,предыдцщий_месяц,13
2,215349,текущий_месяц,10
3,215349,предыдцщий_месяц,12
4,215350,текущий_месяц,13


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600 entries, 0 to 2599
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      2600 non-null   int64 
 1   Период  2600 non-null   object
 2   минут   2600 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 61.1+ KB
None


В таблице 2600 строк без пропусков 

In [8]:
money = pd.read_csv('money - money.csv',sep = ';', decimal = ',')
display(money.head())
print(money.info())

,"id,Прибыль"
0,"215348,""0,98"""
1,"215349,""4,16"""
2,"215350,""3,13"""
3,"215351,""4,87"""
4,"215352,""4,21"""


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1300 entries, 0 to 1299
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id,Прибыль  1300 non-null   object
dtypes: object(1)
memory usage: 10.3+ KB
None


в таблице 1300 строк без пропусков

##  Предобработка данных

In [ ]:
#market_file

#Проверяем на дубликаты столбец ID 
market_file['id'].duplicated().sum() # - дубликатов нет 

#Проверяем текстовые столбы на неявные дубликаты
market_file['Покупательская активность'].value_counts() # нет дубликатов 
market_file['Тип сервиса'].value_counts() # 10 опечаток в слове стандарт 
market_file['Разрешить сообщать'].value_counts() # нет дубликатов
market_file['Популярная_категория'].value_counts() # нет дубликатов

#Убираем неявные дубликаты
market_file['Тип сервиса'] = market_file['Тип сервиса'].str.replace('стандартт', 'стандарт') #исправляем опечатку
market_file['Тип сервиса'].value_counts()

#Делаем id индексом
market_file = market_file.set_index('id')
display(market_file.head())
print(market_file.info())

In [ ]:
#market_money

#Проверяем текстовые столбы на неявные дубликаты
market_money['Период'].value_counts() # нет дубликатов 

#Делаем id индексом
market_money = market_money.set_index('id')
display(market_money.head())


In [ ]:
#market_time

#Проверяем текстовые столбы на неявные дубликаты
market_time['Период'].value_counts() # нет дубликатов, но есть опечатки
market_time['Период'] = market_time['Период'].str.replace('предыдцщий_месяц', 'предыдущий_месяц') #исправляем опечатку
market_time['Период'].value_counts() # 


#Делаем id индексом
market_time = market_time.set_index('id')
display(market_time.head())
print(market_time.info())

In [ ]:
#money
#Проверяем на дубликаты столбец ID 
money['id'].duplicated().sum() # - дубликатов нет 

#Делаем id индексом
money = money.set_index('id')
display(money.head())
print(money.info())

В ходе предобработки были выявлены неявные дубликаты в таблице market_file, вызыванные опечатками в столбце тип сервиса. 
Также были выявлены опечатки в таблице market_time в столбце период, хотя это и неповлияло бы на проект, опечатки были исправлены. 
Данным присвоен корретный тип и в таблицах отсутсвуют пропуски. 
В данных нет явных дубликатов. 


In [ ]:
market_file.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', i).lower() for i in market_file.columns]
market_file.columns = market_file.columns.str.replace(' ', '_')
market_file

## Исследовательский анализ данных

In [ ]:
#market_file
display(market_file.head())
market_file.hist(figsize=(20, 20));

In [ ]:
display(market_file['покупательская_активность'].value_counts())

In [ ]:
display(market_file.describe())

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 15));

sns.histplot(data=market_file, x= 'маркет_актив_6_мес', hue='покупательская_активность', multiple='stack', ax=axes[0,0]);
sns.histplot(data=market_file, x= 'маркет_актив_тек_мес', hue='покупательская_активность', multiple='stack', ax=axes[0,1]);
sns.histplot(data=market_file, x= 'длительность', hue='покупательская_активность', multiple='stack', ax=axes[0,2]);
sns.histplot(data=market_file, x= 'акционные_покупки', hue='покупательская_активность', multiple='stack', ax=axes[1,0]);
sns.histplot(data=market_file, x= 'средний_просмотр_категорий_за_визит', hue='покупательская_активность', multiple='stack', ax=axes[1,1]);
sns.histplot(data=market_file, x= 'неоплаченные_продукты_штук_квартал', hue='покупательская_активность', multiple='stack', ax=axes[1,2]);
sns.histplot(data=market_file, x= 'ошибка_сервиса', hue='покупательская_активность', multiple='stack', ax=axes[0,3]);
sns.histplot(data=market_file, x= 'страниц_за_визит', hue='покупательская_активность', multiple='stack', ax=axes[1,3]);



<div class="alert alert-info">
Почти все количественные показатели ниже у группы пользователей со сниженность активностью. 
</div>

In [ ]:
market_file[['маркет_актив_6_мес','маркет_актив_тек_мес','акционные_покупки','средний_просмотр_категорий_за_визит',
           'неоплаченные_продукты_штук_квартал','ошибка_сервиса' ,'страниц_за_визит']].boxplot(figsize=(15, 10));
plt.title('Ящик с усами для каждого столбца')
plt.xticks(rotation=45) 
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 7));
#Изучим отдельно данные с выбросами
sns.boxplot(data = market_file,y = 'маркет_актив_6_мес',ax=axes[0,0]);
sns.boxplot(data = market_file, y = 'длительность',ax=axes[0,1]);
sns.boxplot(data = market_file, y = 'акционные_покупки',ax=axes[1,0]);
sns.boxplot(data = market_file, y = 'неоплаченные_продукты_штук_квартал',ax=axes[1,1]);



<div class="alert alert-info">

Исправлен вывод гистограмм с разбиенем по таргету, вывод графиков бохплот, в одном месте.
Добавлен вывод круговых диаграмм по категориальным признакам

</div>

market_file.query('маркет_актив_6_мес<2.5')['маркет_актив_6_мес'].count()/len(market_file)*100

Примерно 6% данных  по Маркет_актив_6_мес являются сильно меньше медианы. Данные не будут удаляться, так как это реальные данные компании и их стоит 

Всего 13% данных относятся к группе пользователей, у которых доля акционных покупок больше 50%.
Это необходимо учесть при выборе метрик, так как модели будет не хватать данным по таким покупателям. 

In [ ]:
market_file.query('неоплаченные_продукты_штук_квартал>8')['неоплаченные_продукты_штук_квартал'].count()/len(market_file)*100

Только 1% покупателей имеет в корзине более 8 товаров за три месяца и неоплачивает их

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14, 6));


# Создание фигуры и двух подграфиков
(market_file['покупательская_активность'].value_counts()
 .plot(kind='pie', ax=axes[0], legend=False, autopct='%.2f')
 .set(xlabel='', ylabel='', title='покупательская_активность')
);

(market_file['тип_сервиса'].value_counts()
 .plot(kind='pie', ax=axes[1], legend=False, autopct='%.2f')
 .set(xlabel='', ylabel='', title='тип_сервиса')
);
(market_file['популярная_категория'].value_counts()
 .plot(kind='pie', ax=axes[2], legend=False, autopct='%.2f')
 .set(xlabel='', ylabel='', title='популярная_категория')
);


<div class="alert alert-info">

В данных присутсвует дисбаланс классов. Всего 29% премиум клиентов, против 71% стандарт класса. Следует применить 
стратификацию при разделении выборок. Целевой признак тоже имеет дисбаланс. 
Среди популярных категорий лидируют товары для детей. 
</div>

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 6));


# Создание фигуры и двух подграфиков
(market_file.loc[market_file['тип_сервиса']=='стандарт']['популярная_категория'].value_counts()
 .plot(kind='pie', ax=axes[0], legend=False, autopct='%.2f')
 .set(xlabel='', ylabel='', title='покупательская_активность')
);


# Создание фигуры и двух подграфиков
(market_file.loc[market_file['тип_сервиса']=='премиум']['популярная_категория'].value_counts()
 .plot(kind='pie', ax=axes[1], legend=False, autopct='%.2f')
 .set(xlabel='', ylabel='', title='покупательская_активность')
);


<div class="alert alert-info">

нет существенной разницы между категориями для сегментов стандарт и премиум
</div>


In [ ]:
display(market_money.head())

market_money['Период'].value_counts()
market_money.pivot_table(index='Период',values='Выручка',aggfunc='sum')

In [ ]:
market_money.isna().sum()
market_money.info()

Данные по периодам разнесены равномерно

In [ ]:
display(market_money.describe())
market_money.boxplot(figsize=(10,5));
market_money.query('Выручка>100000')['Выручка'].count()
display(market_money.query('Выручка>100000'))

В данных присутствует 1 строка с выручкой более 100к за текущий месяц, такую строку лучше удалить, так как она одна - ценности для обучения она не принесет, но может снизить его качество. Удалим эту строку позже.

In [ ]:
#продолжение анализа 
market_money.boxplot(figsize=(10,5));
market_money.query('Выручка<1000')['Выручка'].count()
display(market_money.query('Выручка<1000'))

В данных имеются записи о пользователях, которые ничего не купили за прошлый периоды. Позже их нужно будет удалить.  В данных также присутствует много выбросов, стоит это учитывать при обучении. 

In [ ]:

display(market_time.head())
market_time['Период'].value_counts()
market_time.pivot_table(index='Период',values='минут',aggfunc='sum')

Данные распределены равномерно

In [ ]:
market_time.boxplot(figsize=(10,5));

Выбросов в данных нет. 

In [ ]:

# Построение гистограмм по минутам за текущий и прошлый месяцы
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6));
sns.histplot(market_time.loc[market_time['Период']=='текущий_месяц'], bins=10, kde=True,ax=axes[0]); 
axes[0].set_xlabel('текущий_месяц')
axes[0].set_ylabel('Количество');
sns.histplot(market_time.loc[market_time['Период']=='предыдущий_месяц'], bins=10, kde=True,ax=axes[1]);  
axes[1].set_xlabel('предыдущий_месяц')
axes[1].set_ylabel('Количество');



<div class="alert alert-info">
В текущем месяце распределение времени близко к нормальному, в предыдущем месяце распределение имеет два пика,
это может говорить о наличии разных сегментов пользователей

</div>


In [ ]:
market_money.head()

In [ ]:
# Постороение гистрограмм по выручке за последние три месяца
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5));
sns.histplot(market_money.loc[market_money['Период']=='текущий_месяц'], bins=100,ax=axes[0]); 
axes[0].set_xlim(0, 10000)
axes[0].set_xlabel('текущий_месяц')
axes[0].set_ylabel('Количество');
sns.histplot(market_money.loc[market_money['Период']=='предыдущий_месяц'], bins=30, kde=True,ax=axes[1]);  
axes[1].set_xlabel('предыдущий_месяц')
axes[1].set_ylabel('Количество');
sns.histplot(market_money.loc[market_money['Период']=='препредыдущий_месяц'], bins=30, kde=True,ax=axes[2]);  
axes[2].set_xlabel('препредыдущий_месяц')
axes[2].set_ylabel('Количество');


<div class="alert alert-info">
Выручка имеет близкое к нормальному распределение, средние значение выручки на пользователя остается около 5к. 
</div>

<div>
<h3>Вывод</h3>
    В данных присутсвуют выбросы, это может отразится на качестве обучения модели. Стоит учитывать это при выборе метрики качества. Если качество будет слишком низким, потребуется нивилировать данный дисбаланс. 
    
    
- В данных больше пользователей, у которых было более 4 маркетенгкивых активностей и есть группа пользователей, у которых было совсем мало маркт. контакт, менее 2.5
- есть дву группы пользователей, первый у которых акционных покупок менее 40% и вторые - у кого таких покупок 80% и более процентов. Первых существенно больше. Стоит учитывать данный дисбаланс. 
-  Найден пользователь у которого Выручка>100000 за текущий месяц. Возможно это баг и стоит проверить сбор данных, в любом случае, данная строка была удалена из данных, чтобы не ухудшать качество обучения.  
    
<div>

## Объединение таблиц

In [ ]:
market_money.head()
market_money.info()

In [ ]:
display(market_money.head())
market_money['выручка_тек']= market_money.query('Период=="текущий_месяц"')['Выручка']
market_money['выручка_пре'] = market_money.query('Период=="предыдущий_месяц"')['Выручка']
market_money['выручка_пре_пре'] = market_money.query('Период=="препредыдущий_месяц"')['Выручка']

display(market_money.head())
market_money.info()

In [ ]:
# Удаление дубликатов по индексу, оставляя последнюю встреченную строку
market_money = market_money.reset_index()

market_money = market_money.drop_duplicates(subset='id',keep = 'first')
market_money.set_index('id', inplace=True)
display(market_money.head())
market_money.info()


In [ ]:

market_money = market_money.drop(['Период','Выручка'],axis = 1)
market_money.head(5)

In [ ]:
market_money.isna().sum()
#удаление выброса 
market_money = market_money.query('выручка_тек <= 100000')
market_money.info()

In [ ]:
market_time.isna().sum()
market_time.info()

market_time['минут_тек']= market_time.query('Период=="текущий_месяц"')['минут']
market_time['минут_пре'] = market_time.query('Период=="предыдущий_месяц"')['минут']
display(market_time.head())


In [ ]:
# Удаление дубликатов по индексу, оставляя последнюю встреченную строку
market_time = market_time.reset_index()
print(market_time.duplicated(subset='id').sum())

market_time = market_time.drop_duplicates(subset='id')
market_time.info()

In [ ]:
# Вернем индекс обратно
market_time.set_index('id', inplace=True)
market_time.head()

In [ ]:
market_time = market_time.drop(['Период','минут'],axis = 1)
display(market_time.head())

In [ ]:
df = market_file.join(market_time).join(market_money)

display(df.head())
df.isna().sum()

In [ ]:
df=df.fillna(0)
df.isna().sum()

In [ ]:
#удаляем пользователей, у которых покупательская активность меньше 3 месяцев.
drop = df.query('(выручка_пре ==0) or (выручка_пре_пре == 0)').index
df = df.drop(index=drop)
display(df.head())
df.info()

In [ ]:
df.isna().sum()

В ходе объединения был удален пользователь, у которого выручка за текущий месяц превышает 100к, также были удалены 3 пользователя, у которых покупательская активность менее трех месяцев. Пропусков в таблице нет. 

## Корреляционный анализ

In [ ]:
phik_overview = phik_matrix(df) 

In [ ]:
plot_correlation_matrix(
    phik_overview.values,
    x_labels=phik_overview.columns,
    y_labels=phik_overview.index,
    vmin=0, vmax=1, color_map='Blues',
    title=r'correlation $\phi_K$',
    fontsize_factor=1.5,
    figsize=(20, 15)
) 


В данных нет мультиколинераности, так как все коэф. ниже 0.9 

In [ ]:
pd.plotting.scatter_matrix(df, figsize=(20, 20));

В данных есть линейные зависомомсти между текущей выручкой и выручкой за прошлый месяца. Есть группы
пользователей у которых эта зависимость сильно выражена. 

In [ ]:
print(df.info())
df.head()


In [ ]:
sns.scatterplot(data=df, x='id', y='акционные_покупки', hue='покупательская_активность');

<div class="alert alert-info">
Есть две группы пользователей, одна группа предпочитает покупать только по акциям,
у второй группы доля акционных покупок менее 40% 
Также стоит отметить, что почти все пользователи c прежним уровнем активности практически не ориентируются на акционные товары; 
</div>

## Использование пайплайнов


In [ ]:
target = df['покупательская_активность']
label_mapping = {'Снизилась': 1, 'Прежний уровень': 0}
label_encoder = LabelEncoder()
label_encoder.fit_transform(target.map(label_mapping))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['покупательская_активность'], axis=1),
    df['покупательская_активность'],
    test_size = TEST_SIZE, 
    random_state = RANDOM_STATE,
    stratify = df['покупательская_активность'])

X_train.shape, X_test.shape

In [ ]:
# создаём списки с названиями признаков
ohe_columns = ['тип_сервиса', 'разрешить_сообщать','популярная_категория']
num_columns = ['маркет_актив_6_мес', 'маркет_актив_тек_мес','длительность','акционные_покупки',
               'средний_просмотр_категорий_за_визит','неоплаченные_продукты_штук_квартал','ошибка_сервиса',
              'страниц_за_визит','минут_тек','минут_пре','выручка_тек','выручка_пре','выручка_пре_пре']


In [ ]:

# создаём пайплайн для подготовки признаков из списка ohe_columns: заполнение пропусков и OHE-кодирование
# SimpleImputer + OHE
ohe_pipe = Pipeline(
    [('simpleImputer_ohe', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
     ('ohe', OneHotEncoder(drop='first', sparse_output=False))
    ]
)

# создаём общий пайплайн для подготовки данных
data_preprocessor = ColumnTransformer(
    [('ohe', ohe_pipe, ohe_columns),
     ('num', MinMaxScaler(), num_columns)
    ], 
    #remainder='passthrough'
)

# создаём итоговый пайплайн: подготовка данных и модель
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
])

param_grid = [
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 5),
        'models__max_features': range(2, 5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    
    # словарь для модели KNeighborsClassifier()
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },

    # словарь для модели LogisticRegression()
    {
        'models': [
            LogisticRegression(
                random_state=RANDOM_STATE, 
                solver='liblinear', 
                penalty='l2',  # Изменили на 'l2'
                max_iter=200   # Добавили параметр max_iter
            )
        ],
        'models__C': range(1, 5),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    
    # словарь для модели SVC()
    {
        'models': [
            SVC(
                random_state=RANDOM_STATE,
            )
        ],
        'models__kernel': ['poly', 'rbf'],
        'models__degree':range(1,8),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    }
]

randomized_search = RandomizedSearchCV(
    pipe_final,
    param_grid,
    cv=5,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1,
    n_iter=10
)


randomized_search.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print('Метрика лучшей модели по кросс валидационной выборке:', randomized_search.best_score_)



In [ ]:

# Предсказание на тестовой выборке
y_pred = randomized_search.best_estimator_.predict(X_test)
y_pred_proba = randomized_search.best_estimator_.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f'Метрика ROC-AUC на тестовой выборке: {roc_auc:.2f}')

<div class="alert alert-info">
Обновила
</div>

In [ ]:


# Вычисление матрицы ошибок
conf_matrix = confusion_matrix(y_test, y_pred)


# Визуализация матрицы ошибок
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Матрица ошибок')
plt.show()


In [ ]:
best_model= randomized_search.best_estimator_
X_train_transformed = pd.DataFrame(best_model.named_steps['preprocessor'].transform(X_train), 
                                   columns=best_model.named_steps['preprocessor'].get_feature_names_out())
X_test_transformed = pd.DataFrame(best_model.named_steps['preprocessor'].transform(X_test), 
                                   columns=best_model.named_steps['preprocessor'].get_feature_names_out())
display(X_test_transformed.head())

Для оценки качества была выбрана метрика roc_auc, так как она более устойчива к выбросам в данных и к дисбалансам классов. 
Модель с наиллучшей оценкой качества оказалась LogisticRegressio и оценкой 0.89

<div class="alert alert-info">
Обновила
</div>

## Анализ важности признаков

In [ ]:
best_model = randomized_search.best_estimator_

# Создание объяснителя SHAP с указанием имен признаков
explainer = shap.LinearExplainer(best_model.named_steps['models'], X_train_transformed, feature_perturbation='interventional')

# Получение значений SHAP
shap_values = explainer(X_test_transformed)


# Построение графика
shap.summary_plot(shap_values, X_test_transformed, plot_type='bar', max_display=16, plot_size=(10, 8))


Самый главный признак для модели это кол-во страниц за визит, чем выше значение, тем больше вероятность покупки
и как следствие проставление целевого признака - прежняя покупательская активность. 
Далее значемые признаки: 
    
    - кол-во минут в текущем месяце (10)
    - Неоплаченные_продукты_штук_квартал (7)
    - Выручка в тек месяце (12)
    - Кол-во минут в прошлом месяце (11)
    - Кол-во ошибок сервиса (8)
    - Средний_просмотр_категорий_за_визит (6)
    - Маркет_актив_тек_мес (3)
    
Малозначемые признаки: 
    
    - Тип сервиса (0)
    - Длительность (4)
    - Разрешить сообщать 


<div class="alert alert-info">
Самый главный признак для модели это кол-во страниц за визит, чем выше значение, тем больше вероятность покупки и как следствие проставление целевого признака - прежняя покупательская активность. Далее значемые признаки:

- кол-во страниц за визит
- ср. просмотр категорий за визит
- кол-во минут в прошлом месяце

Малозначемые признаки:
- Разрешить сообщать 
- Популярная категория Товары для детей
- Длительность
</div>

In [ ]:
shap.plots.beeswarm(shap_values, max_display=16) 

- Чем выше кол-во минут проведенных на сайте в текущем месяце, тем больше вероятность, что
у пользователя сохраниться прежний статус.
- Неоплаченные_продукты_штук_квартал - чем выше кол-во неоплаченных товаров в корзине, тем выше вероятность,что
у пользователя сохраниться прежний статус.
- Чем чаще пользователь сталкивается с ошибками сервиса, тем выше вероятность снижения активности 
- Чем больше категорий смотрит пользователь за визит, тем выше вероятность снижения активности 
- Чем выше маркт активность в текущем месце,тем выше вероятность,что
у пользователя сохраниться прежний статус.

<div class="alert alert-info">
    
- высокие значения кол-ва посещенных страниц, ср. кол-ва категорий за посещение, минут на сайте, а также маркт активности
за 6 месяцев - склоняют модель к таргету 0
    
- выоские значения в признаках: неоплаченные продукты, акционные покупки, вырчка за прошлый есяц - склоняют модель 
к таргету 1, то есть к снижению активности.  

</div>

In [ ]:
shap.plots.waterfall(shap_values[0]) 


Выводы для бизнеса: 

    - сократить кол-во ошибок на сайте
    - присылать пользователю таргетированный контент, чтобы стимулировать изучение ассортимента на сайте, посещать больше страниц
    - вовлекать пользователя маркет активностями пользователей с низким временемна сайте в тек месяце,чем выше время на сайе - тем больше шансов на покупку 
    - стимулировать покупку, для пользователей с брошенными корзинами 


## Сегментация покупателей


Варианты сегментов: 
- пользователи с брошенными корзинами 
- пользователи у которых наблюдается снижение времени на сайте 
- пользователи с высоким показателем ошибок на сайте 

In [ ]:
df.head()
money.head()

In [ ]:
df_money= df.join(money)
df_money.info()

In [ ]:
#Отбираем клиентов, которые приносят больше всего прибыли
df_money.sort_values(by='Прибыль')
df_money['Прибыль'].describe()
df_prof = df_money.loc[df_money['Прибыль']>4.6]

#Отбираем среди них клиентов, у которых выская доля использования акционных покупок
df_money['акционные_покупки'].describe()
df_prof =  df_prof.loc[df_money['акционные_покупки']>0.3] #клиенты которые приносят больше всего прибыли, и имеют высокий риск снижения
df_new = df_money.loc[df_money['акционные_покупки']<=0.3]#клиенты которые приносят больше всего прибыли, и имеют прежнюю активность 




<div class="alert alert-info">
    
Согласно анализу модели и диаграммы рассеиния, чем больше клиент покупает по акции, тем сильнее это оказывает в моделе 
соответсвие снижению активности. Таким образом мы выделели сегмент покупателей, с наибольшей выручкой, которые 
с выс
    

</div>

In [ ]:
df_new.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 15));

sns.histplot(data=df_prof, x= 'маркет_актив_6_мес', multiple='stack', ax=axes[0,0],color='blue',alpha=0.5,label='df_prof');
sns.histplot(data=df_new, x= 'маркет_актив_6_мес', multiple='stack', ax=axes[0,0],color= 'red',alpha=0.5,label='df_new');

sns.histplot(data=df_prof, x= 'маркет_актив_тек_мес',  multiple='stack', ax=axes[0,1],color='blue',alpha=0.5,label='df_prof');
sns.histplot(data=df_new, x= 'маркет_актив_тек_мес', multiple='stack', ax=axes[0,1],color= 'red',alpha=0.5,label='df_new');
             
sns.histplot(data=df_prof, x= 'длительность', multiple='stack', ax=axes[0,2],color='blue',alpha=0.5,label='df_prof');
sns.histplot(data=df_new, x= 'длительность', multiple='stack', ax=axes[0,2],color= 'red',alpha=0.5,label='df_new');             
                          
             
sns.histplot(data=df_prof, x= 'средний_просмотр_категорий_за_визит', multiple='stack', ax=axes[1,0],color='blue',alpha=0.5,label='df_prof');
sns.histplot(data=df_new, x= 'средний_просмотр_категорий_за_визит', multiple='stack', ax=axes[1,0],color= 'red',alpha=0.5,label='df_new');             
             
sns.histplot(data=df_prof, x= 'неоплаченные_продукты_штук_квартал', multiple='stack', ax=axes[1,1],color='blue',alpha=0.5,label='df_prof');
sns.histplot(data=df_new, x= 'неоплаченные_продукты_штук_квартал', multiple='stack', ax=axes[1,1],color= 'red',alpha=0.5,label='df_new');             
             
sns.histplot(data=df_prof, x= 'ошибка_сервиса',  multiple='stack', ax=axes[0,3],color='blue',alpha=0.5,label='df_prof');
sns.histplot(data=df_new, x= 'ошибка_сервиса', multiple='stack', ax=axes[0,3],color= 'red',alpha=0.5,label='df_new');             
             
sns.histplot(data=df_prof, x= 'страниц_за_визит', multiple='stack', ax=axes[1,2],color='blue',alpha=0.5,label='df_prof');
sns.histplot(data=df_new, x= 'страниц_за_визит', multiple='stack', ax=axes[1,2],color= 'red',alpha=0.5,label='df_new');




<div class="alert alert-info">
у выбранного сегмента с высоким риском снижения, распределение графика маркет активности за 6 месяцев смещено влево,
это значит, что больше пользователей имеет меньше среднего точек касания с маркетингом. 
Также у категории с риском снижения в среднем меньше кол-ва просмотров категорий страниц за сеанс и 
кол-во просмотров страниц. 

</div>


In [ ]:
display(df_prof.describe())
df_new.describe()



<div class="alert alert-info">
Также у выбранного сегмента больше неоплаченных товаров в корзине, чем у альтернативного.

</div>


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6));


# Создание фигуры и двух подграфиков

(df_new['популярная_категория'].value_counts()
 .plot(kind='pie', ax=axes[0], legend=False, autopct='%.2f')
 .set(xlabel='', ylabel='', title='популярная_категория')
);
(df_prof['популярная_категория'].value_counts()
 .plot(kind='pie', ax=axes[1], legend=False, autopct='%.2f')
 .set(xlabel='', ylabel='', title='популярная_категория_снижение')
);


   

<div class="alert alert-info">
Чтобы активизировать покупательскую активность у выбранного семента можно:
    
    - прислать письмо и пуш уведомление о брошенных товарах в корзине, предложить скидку на всю корзину при ппокупке в этом месяце. 
    - при покупке товаров на минимальную сумму в этом месяце, дать промокод со скидкой на покупку в след месяце 
    по самой популярной категории для данного сегмента - Товары для детей
 
</div>



<div class="alert alert-info">
Чтобы уменьшеть снижение покупательской активности в целом, нужно поднять время провождение на сайте и глубину просмотра. 
Для этого можно использовать на всех пользователях у которых модель предсказывает снижение активности:  
    
- рассылки с сезонными подборки (отпуска, учеба, спорт и тд)
- настроить блок с рекомендательной полкой - чтобы пользователь мог ходить по разным категориям 
- напоминание о покупке о товарах первой необходимости исходя из частоты покупок. 
</div>



Предлагается взять сегмент, у которых: 
    
- в корзине лежит более 3 неоплаченных товаров за последний квартал
- кол-во минут на сайте в текущем месяце менее 13
У большинства этих пользователей покупательская активность уже помечена как сниженная, у остальной части она может 
снижаться в следующих месяцах. 
Долья прибыли таких пользователей более 30% от общей прибыли. 

- Для данных пользователей можно сформировать специальное прделожение на товары находящиеся в корзине, а 
также предложения по товарам в любимой категории. 
- По данным пользователем можно увеличить кол-во маркет активностей, с целью увеличения времени проведенного сайте


## Общий вывод


<div class="alert alert-info">
Вывод обновлен
</div>


Цель: выявить сегмент покупателей, у которых предполагается снижение покупательской активности, 
    чтобы сделать им персональные предложения.

Цель проекта: Обучить 4 модели. Выбрать лучшую модель с наиболее высокой метрикой качества. Провести анализ важности признаков лучшей модели и сформировать сегмент пользователей и предложить способы увеличения покупательской активности для этого сегмента.

Результаты проекта: 


    1. загружены 4 датасета
    2. проведена предобработка данных, исправлены опечатки, необходимо перепроверить сбор данных, 
    возможно это опечатки в системе, то есть баги.
    3. Проведено исследовательский анализ, 
        - удалены 3 пользователя, у которых за последние 3 месяца не было покупок.
        - выявлена и удалена одна строка с крайне высоким сверхвыручкой по пользователю за тек период.
        Вероятно это ошибка системы, необходимо перероверить даные по пользователю.
    4. Объединены 3 датасета
    5. Проведен корреляционный анализ,в результате которого не было выявлено мультиколлиниарности и кор-ции
     с целевыми признаком. 
    6. Обучено 4 модели. Для поиска лучшей модели был реализован пайплайн, который использует  кодировщик 
    и 2 скайлера, также пайплайн использует авматический подбор гиперпараметров с кроссвлидацией из 5 блоков.
    Лучшая модель оказалась Модель линейной регрессии. 
    7.Проведен анализ важности признаков. Наиболее важными оказались признаки: 
        - кол-во страниц за визит
        - Средний_просмотр_категорий_за_визит
        - кол-во минут в прошлом месяце и в текущем

    Малозначемые признаки:

        - Разрешить сообщать 
        - Длительность
        - Популярная категория - Товары для детей
           
    8. Проведен анализ сегмента. 
    
        - Был выбран сегмент пользователей, который приносит больше всего прибыли компании и по важному для модели
        признаку отфлитрованы пользователи с тенденцией к снижению активности. Далее этот сегмент сравнивался с такими же пользователями, но с меньшим риском снижения активности. 
       Пользователи в выборнном сегменте, проводят меньше времени на сайте и просматривают меньше категорий, а также имеют меньшее соприкосновение с маркетингом за последние 6 месяцев и имеют больше неоплаченных товаров в корзине. 
       Чтобы повысить их актиновсть предлагается:
               
    - прислать письмо и пуш уведомление о брошенных товарах в корзине, предложить скидку на всю корзину при ппокупке в этом месяце. 
    - при покупке товаров на минимальную сумму в этом месяце, дать промокод со скидкой на покупку в след месяце 
    по самой популярной категории для данного сегмента - Товары для детей
       
